In [1]:
import os
import fitsio
import subprocess
import meds
from metadetect.fitting import fit_mbobs_list_wavg
import ngmix
from ngmix.medsreaders import NGMixMEDS, MultiBandNGMixMEDS
import more_itertools
import joblib

In [2]:
def _download(fname):
    os.makedirs("./data", exist_ok=True)
    cmd = """\
    rsync \
            -av \
            --password-file $DES_RSYNC_PASSFILE \
            ${DESREMOTE_RSYNC_USER}@${DESREMOTE_RSYNC}/%s \
            ./data/%s
    """ % (fname, os.path.basename(fname))
    subprocess.run(cmd, shell=True, check=True)
    return "./data/%s" % os.path.basename(fname)

In [3]:
fnames = fitsio.read("fnames.fits")

In [4]:
mfiles = [
    _download(os.path.join(fnames["PATH"][i], fnames["FILENAME"][i]))
    for i in range(fnames.shape[0])
]


    DES rsync daemon

receiving incremental file list

sent 19 bytes  received 98 bytes  78.00 bytes/sec
total size is 1050166080  speedup is 8975778.46

    DES rsync daemon

receiving incremental file list

sent 19 bytes  received 98 bytes  234.00 bytes/sec
total size is 940207680  speedup is 8035963.08

    DES rsync daemon

receiving incremental file list

sent 19 bytes  received 98 bytes  78.00 bytes/sec
total size is 988254720  speedup is 8446621.54

    DES rsync daemon

receiving incremental file list

sent 19 bytes  received 98 bytes  234.00 bytes/sec
total size is 962167680  speedup is 8223655.38


In [5]:
def _fit(jinds, mpths):
    m = None
    try:
        m = MultiBandNGMixMEDS([NGMixMEDS(pth) for pth in mpths])

        mbobs_list = [
            m.get_mbobs(i)
            for i in jinds
        ]       
        fitter = ngmix.gaussmom.GaussMom(1.2)

        return fit_mbobs_list_wavg(
            mbobs_list=mbobs_list,
            fitter=fitter,
            bmask_flags=2**30,
            symmetrize=False,
        )
    finally:
        if m is not None:
            for _m in m.mlist:
                _m.close()

In [6]:
m = meds.MEDS(mfiles[0])
n = m.size
m.close()

job_inds = list(more_itertools.chunked(range(n), n//99))

jobs = []
for jinds in job_inds:
    jobs.append(
        joblib.delayed(_fit)(jinds, mfiles)
    )

In [7]:
with joblib.Parallel(n_jobs=10, backend="multiprocessing", verbose=100) as exc:
    res = exc(jobs)

[Parallel(n_jobs=10)]: Using backend MultiprocessingBackend with 10 concurrent workers.
[Parallel(n_jobs=10)]: Done   1 tasks      | elapsed:  1.9min
[Parallel(n_jobs=10)]: Done   2 tasks      | elapsed:  2.0min
[Parallel(n_jobs=10)]: Done   3 tasks      | elapsed:  2.0min
[Parallel(n_jobs=10)]: Done   4 tasks      | elapsed:  2.2min
[Parallel(n_jobs=10)]: Done   5 tasks      | elapsed:  2.2min
[Parallel(n_jobs=10)]: Done   6 tasks      | elapsed:  3.3min
[Parallel(n_jobs=10)]: Done   7 tasks      | elapsed:  3.5min
[Parallel(n_jobs=10)]: Done   8 tasks      | elapsed:  3.6min
[Parallel(n_jobs=10)]: Done   9 tasks      | elapsed:  3.6min
[Parallel(n_jobs=10)]: Done  10 tasks      | elapsed:  3.6min
[Parallel(n_jobs=10)]: Done  11 tasks      | elapsed:  4.7min
[Parallel(n_jobs=10)]: Done  12 tasks      | elapsed:  4.7min
[Parallel(n_jobs=10)]: Done  13 tasks      | elapsed:  4.8min
[Parallel(n_jobs=10)]: Done  14 tasks      | elapsed:  4.8min
[Parallel(n_jobs=10)]: Done  15 tasks      |

In [8]:
import numpy as np

res = np.concatenate(res, axis=0)

In [9]:
msk = res["wmom_flags"] == 0

import proplot as pplt


fig, axs = pplt.subplots()
axs.hexbin(
    np.log10(res[msk]["wmom_s2n"]),
    res[msk]["wmom_T_ratio"],
    extent=(0, 4, 0.5, 1.25),
    gridsize=100,
    norm="log",
    cmap="rocket",
)
axs.format(xlim=(0, 4), ylim=(0.5, 1.25))

np.median(res[msk & (res["wmom_s2n"] > 1e3)]["wmom_T_ratio"])-1

ModuleNotFoundError: No module named 'proplot'

In [ ]:
import esutil

s2n = 0.0
for i in esutil.pbar.PBar(list(range(m.size))):
    mbobs = m.get_mbobs(i, weight_type="uberseg")
    s2n += mbobs.get_s2n()
s2n /= m.size

In [ ]:
import numpy as np

last = None
for i in range(m.size):
    for j in range(m["ncutout"][i]):
        if last is None:
            assert m["psf_start_row"][i, j] == 0
        else:
            assert m["psf_start_row"][i, j] == last
        last = m["psf_start_row"][i, j] + m["psf_row_size"][i, j] * m["psf_col_size"][i, j]
        
        assert (
            (m["psf_cutout_row"][i, j] >= 0)
            & ((m["psf_cutout_row"][i, j] <= m["psf_row_size"][i, j]))
        )
        
        assert (
            (m["psf_cutout_col"][i, j] >= 0)
            & ((m["psf_cutout_col"][i, j] <= m["psf_col_size"][i, j]))
        )

In [ ]:
with fitsio.FITS(pth) as f:
    print(f["psf"])